<a href="https://colab.research.google.com/github/Skrishna04/Cancer_subtypes/blob/Rhea/XGB_with_1196.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import os
import re
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix
from xgboost import XGBClassifier



In [3]:
# ------------ USER CONFIG ------------
expr_csv_path = r"/content/BRCA_assay_1.csv"
random_state = 42
top_k_features = 500
test_size = 0.2


In [4]:
# 0) basic checks
if not os.path.exists(expr_csv_path):
    raise FileNotFoundError(f"Expression CSV not found: {expr_csv_path}")

# 1) Load expression matrix (R-exported CSV usually rows=genes, cols=samples)
expr = pd.read_csv(expr_csv_path, index_col=0)
print("Raw CSV shape (rows x cols):", expr.shape)

# 2) Ensure samples are rows and features are columns
# If rows >> cols, assume rows are genes and transpose
if expr.shape[0] > expr.shape[1]:
    expr = expr.T
    print("Transposed expression to samples x features. New shape:", expr.shape)
else:
    print("Expression already samples x features:", expr.shape)



Raw CSV shape (rows x cols): (2700, 1196)
Transposed expression to samples x features. New shape: (1196, 2700)


In [7]:
# Normalize sample IDs to strings
expr.index = expr.index.astype(str).str.strip()

# 3) Infer TCGA sample type code from barcode
def get_sample_type_code(barcode: str):
    if not isinstance(barcode, str):
        return None
    parts = barcode.split('-')
    if len(parts) >= 4 and len(parts[3]) >= 2:
        return parts[3][:2]
    m = re.search(r'-(\d{2}[A-Z0-9])-', barcode)
    if m:
        return m.group(1)[:2]
    return None

sample_type_map = {
    '01': 'Primary_Tumor',
    '02': 'Recurrent_Tumor',
    '03': 'Primary_Blood_Derived_Tumor',
    '04': 'Metastatic',
    '05': 'Additional_Metastatic',
    '06': 'Metastatic_Recurrent',
    '07': 'Xenograft',
    '08': 'Cell_Line',
    '09': 'Primary_Microdissected',
    '10': 'Blood_Derived_Normal',
    '11': 'Solid_Tissue_Normal',
    '12': 'Buccal_Cell_Normal'
}

meta = pd.DataFrame(index=expr.index)
meta['sample_type_code'] = [get_sample_type_code(b) for b in expr.index]
meta['sample_type'] = meta['sample_type_code'].map(sample_type_map).fillna('Unknown')

print("Inferred sample types (top counts):")
print(meta['sample_type'].value_counts().head(20))



Inferred sample types (top counts):
sample_type
Primary_Tumor           1076
Solid_Tissue_Normal      113
Metastatic_Recurrent       7
Name: count, dtype: int64


In [8]:
# 4) Make binary label: Primary_Tumor vs Normal (others -> Other)
meta['binary_label'] = meta['sample_type'].apply(
    lambda x: 'Primary_Tumor' if x == 'Primary_Tumor' else ('Normal' if x in ('Solid_Tissue_Normal', 'Blood_Derived_Normal') else 'Other')
)
print("\nBinary label counts:")
print(meta['binary_label'].value_counts())




Binary label counts:
binary_label
Primary_Tumor    1076
Normal            113
Other               7
Name: count, dtype: int64


In [9]:
# 5) Keep only Primary_Tumor and Normal samples for a clear binary classification
keep_mask = meta['binary_label'].isin(['Primary_Tumor', 'Normal'])
expr = expr.loc[keep_mask]
y = meta.loc[keep_mask, 'binary_label']
print("\nAfter filtering to Primary_Tumor vs Normal:")
print("Samples:", expr.shape[0], "Features:", expr.shape[1])
print("Label distribution:\n", y.value_counts())

if expr.shape[0] < 10:
    raise SystemExit("Too few samples after filtering to Primary_Tumor vs Normal. Stop.")




After filtering to Primary_Tumor vs Normal:
Samples: 1189 Features: 2700
Label distribution:
 binary_label
Primary_Tumor    1076
Normal            113
Name: count, dtype: int64


In [10]:
# 6) Encode labels
le = LabelEncoder()
y_enc = le.fit_transform(y)  # 0/1 encoded
print("\nLabel mapping:", dict(zip(le.classes_, le.transform(le.classes_))))




Label mapping: {'Normal': np.int64(0), 'Primary_Tumor': np.int64(1)}


In [12]:
# 7) Handle missing values in expression (simple median imputation)
if expr.isnull().any().any():
    expr = expr.fillna(expr.median(numeric_only=True))
    print("Filled missing values with column medians.")



In [13]:
# 8) Train/test split (stratify on label)
strat = y_enc if len(np.unique(y_enc))>1 and min(np.bincount(y_enc))>=2 else None
X_train, X_test, y_train, y_test = train_test_split(
    expr, y_enc, test_size=test_size, random_state=random_state, stratify=strat
)
print("\nTrain shape:", X_train.shape, "Test shape:", X_test.shape)




Train shape: (951, 2700) Test shape: (238, 2700)


In [15]:
# 9) Train baseline XGBoost on all features (use training set only)
xgb = XGBClassifier(
    n_estimators=200,
    max_depth=6,
    use_label_encoder=False,
    eval_metric="logloss",
    random_state=random_state,
    verbosity=1
)
xgb.fit(X_train, y_train)

# Evaluate XGBoost on test
y_pred_xgb = xgb.predict(X_test)
acc_xgb = accuracy_score(y_test, y_pred_xgb)
print("\nXGBoost Test Accuracy: {:.4f}".format(acc_xgb))
print("XGBoost Classification Report:")
print(classification_report(y_test, y_pred_xgb, target_names=le.classes_))
if len(le.classes_) == 2:
    y_proba_xgb = xgb.predict_proba(X_test)[:, 1]
    try:
        print("XGBoost ROC AUC:", roc_auc_score(y_test, y_proba_xgb))
    except Exception as e:
        print("Could not compute ROC AUC:", e)



/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [08:46:47] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



XGBoost Test Accuracy: 0.9832
XGBoost Classification Report:
               precision    recall  f1-score   support

       Normal       0.95      0.87      0.91        23
Primary_Tumor       0.99      1.00      0.99       215

     accuracy                           0.98       238
    macro avg       0.97      0.93      0.95       238
 weighted avg       0.98      0.98      0.98       238

XGBoost ROC AUC: 0.9991911021233569


In [16]:
# 10) Get feature importances from XGBoost (trained on training data)
feat_imp = pd.Series(xgb.feature_importances_, index=X_train.columns).sort_values(ascending=False)
k = min(top_k_features, X_train.shape[1])
top_features = feat_imp.index[:k].tolist()
print(f"\nSelected top {k} features from XGBoost importances.")




Selected top 500 features from XGBoost importances.


In [17]:
# 11) Subset data to top features and scale for Logistic Regression
X_train_sel = X_train[top_features].copy()
X_test_sel  = X_test[top_features].copy()

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_sel)
X_test_scaled  = scaler.transform(X_test_sel)



In [18]:
# 12) Train Logistic Regression using top features
# Use balanced class_weight if imbalance detected (heuristic)
cls_counts = np.bincount(y_train)
use_balanced = (cls_counts.min() < 0.5 * cls_counts.max())
lr = LogisticRegression(max_iter=2000, solver='liblinear', class_weight='balanced' if use_balanced else None, random_state=random_state)
lr.fit(X_train_scaled, y_train)

# Evaluate LR on test set
y_pred_lr = lr.predict(X_test_scaled)
acc_lr = accuracy_score(y_test, y_pred_lr)
print("\nLogistic Regression Test Accuracy: {:.4f}".format(acc_lr))
print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_lr, target_names=le.classes_))

# ROC AUC for LR (binary)
if len(le.classes_) == 2:
    y_proba_lr = lr.predict_proba(X_test_scaled)[:, 1]
    try:
        print("Logistic Regression ROC AUC:", roc_auc_score(y_test, y_proba_lr))
    except Exception as e:
        print("Could not compute ROC AUC:", e)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_lr)
print("\nLogistic Regression Confusion Matrix (rows=true, cols=pred):\n", cm)




Logistic Regression Test Accuracy: 0.9790
Logistic Regression Classification Report:
               precision    recall  f1-score   support

       Normal       0.82      1.00      0.90        23
Primary_Tumor       1.00      0.98      0.99       215

     accuracy                           0.98       238
    macro avg       0.91      0.99      0.95       238
 weighted avg       0.98      0.98      0.98       238

Logistic Regression ROC AUC: 0.9977755308392315

Logistic Regression Confusion Matrix (rows=true, cols=pred):
 [[ 23   0]
 [  5 210]]


In [19]:
# 13) 5-fold CV on training data for LR
cv_scores = cross_val_score(lr, X_train_scaled, y_train, cv=5, scoring='accuracy')
print("\nLogistic Regression 5-fold CV accuracy on training set: mean={:.4f}, std={:.4f}".format(cv_scores.mean(), cv_scores.std()))




Logistic Regression 5-fold CV accuracy on training set: mean=0.9632, std=0.0047


In [20]:
# 14) Show top 20 features chosen
print("\nTop 20 features (by XGBoost importance):")
for i, f in enumerate(top_features[:20], 1):
    print(f"{i}. {f} (importance={feat_imp.loc[f]:.6f})")

# 15) Optionally save top features and models (uncomment if desired)
# X_train_sel.to_csv('X_train_top_features.csv', index=True)
# X_test_sel.to_csv('X_test_top_features.csv', index=True)
# import joblib
# joblib.dump(lr, 'logistic_regression_top500.pkl')
# joblib.dump(xgb, 'xgboost_full.pkl')




Top 20 features (by XGBoost importance):
1. MMP11 (importance=0.340941)
2. BMX (importance=0.137108)
3. FMO2 (importance=0.112306)
4. CPA1 (importance=0.084823)
5. NNAT (importance=0.068080)
6. LIMS2 (importance=0.052927)
7. CXCL2 (importance=0.026616)
8. ATP1A2 (importance=0.024162)
9. TRPM3 (importance=0.023639)
10. ANGPT4 (importance=0.018777)
11. SCN4A (importance=0.016127)
12. WDR62 (importance=0.011789)
13. IBSP (importance=0.011311)
14. CLCA4 (importance=0.010420)
15. FOXP3 (importance=0.008316)
16. CLDN11 (importance=0.006822)
17. GYG2 (importance=0.006101)
18. SCT (importance=0.005877)
19. MAOB (importance=0.005012)
20. OXT (importance=0.003402)

Pipeline finished.
